In [1]:
import os
os.chdir("..")

In [2]:
import torch
from src.dataset import dataset, collate_fn
from src.config import Config
from src.trainer import Model

from athena.utils.progbar import ProgbarCallback
import pytorch_lightning as pl

In [3]:
config = Config(
    DATA_DIR="data",
    IMG_SIZE=640,
    MIN_IMG_SIZE=320,
    BATCH_SIZE=10,
    MIDAS_LOSS_WEIGHT=0,
    YOLO_LOSS_WEIGHT=1,
    EPOCHS=10
)
train_dataset = dataset(config)
loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=config.BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_fn
)

test_dataset = dataset(config, train=False)
loader_test = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=config.BATCH_SIZE,
    collate_fn=collate_fn
)

Caching labels data/labels.npy (3161 found, 0 missing, 42 empty, 0 duplicate, for 3203 images): 100%|██████████| 3203/3203 [00:00<00:00, 10969.79it/s]
Caching labels data/labels.npy (311 found, 0 missing, 7 empty, 0 duplicate, for 318 images): 100%|██████████| 318/318 [00:00<00:00, 3952.41it/s]


In [4]:
model = Model(config, len(loader), 4, train_dataset.yolo_labels)

Model Summary: 222 layers, 6.15399e+07 parameters, 6.15399e+07 gradients


In [5]:
log_dir = "./logs"
name = "Yolo"
tensorboard_logger = pl.loggers.TensorBoardLogger(
    log_dir, name="", version=name, default_hp_metric=False
)
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath=os.path.join(
        log_dir, 
        name,
        "checkpoints",
        "last",
    ),
)

In [6]:
pl.seed_everything(0)
trainer = pl.Trainer(
    max_epochs=config.EPOCHS,
    gpus=1,
    logger=tensorboard_logger,
    checkpoint_callback=checkpoint_callback,
    callbacks=[ProgbarCallback()],
    progress_bar_refresh_rate=20,
)



Global seed set to 0
GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [7]:
trainer.fit(
    model,
    train_dataloader=loader,
    val_dataloaders=loader_test
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type    | Params
---------------------------------
0 | yolo | Darknet | 61.5 M
---------------------------------
61.5 M    Trainable params
0         Non-trainable params
61.5 M    Total params
246.160   Total estimated model params size (MB)
/opt/conda/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
/workspaces/8th-sem-project/src/yolov3/eva5_helper.py:138: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729096996/work/torch/csrc/utils/python_arg_parser.cpp:8

RuntimeError: CUDA out of memory. Tried to allocate 126.00 MiB (GPU 0; 5.80 GiB total capacity; 3.58 GiB already allocated; 167.75 MiB free; 3.59 GiB reserved in total by PyTorch)